# Yolo V3 Eye Detection (Using Open Images V4 Subset)

(Data available from https://www.figure-eight.com/dataset/open-images-annotated-with-bounding-boxes/)

In [2]:
from toolz.curried import *
from funcy import compose, partial, flatten
from os import listdir as ls
from os.path import join, expanduser
import pandas as pd

DIR = expanduser('/media/rcdilorenzo/datahouse/openimagesv4')

data_frame = compose(pd.read_csv, partial(join, DIR))

class_desc = pd.read_csv(join(DIR, 'class-descriptions-boxable.csv'), header=0,
                         names=['LabelName', 'Description'])

# Annotation bounding boxes (train set)
train_ann_bbox      = data_frame('train-annotations-bbox.csv')
# Label w/ confidence by image id (train set)
train_ann_label_box = data_frame('train-annotations-human-imagelabels-boxable.csv')
# Image name with URL (train set)
train_img_box       = data_frame('train-images-boxable.csv')
# Image metadata (e.g. copyright, origin, author, rotation amount) (train set)
train_img_box_rot   = data_frame('train-images-boxable-with-rotation.csv')

# Annotation bounding boxes (validation set)
val_ann_bbox       = data_frame('validation-annotations-bbox.csv')
# Label w/ confidence by image id (validation set)
val_ann_label_box  = data_frame('validation-annotations-human-imagelabels-boxable.csv')
# Image metadata (e.g. copyright, origin, author, rotation amount) (validation set)
val_img_rot        = data_frame('validation-images-with-rotation.csv')

# Annotation bounding boxes (test set)
test_ann_bbox       = data_frame('test-annotations-bbox.csv')
# Label w/ confidence by image id (test set)
test_ann_label_box  = data_frame('test-annotations-human-imagelabels-boxable.csv')
# Image metadata (e.g. copyright, origin, author, rotation amount) (test set)
test_img_rot        = data_frame('test-images-with-rotation.csv')

With these basic imports, the goal is to filter only on the "Human eye" class to build the appropriate model.

In [6]:
LABEL = class_desc[class_desc.Description == 'Human eye'].iloc[0].at['LabelName']
LABEL

'/m/014sv8'

Because this training will be performed on the freely downloadable subset of Open Images V4 (without Google Cloud bucket transfer fees), the annotation metadata needs to not only be filtered on the human eye class but also based on what images are actually included in the subset.

In [11]:
TRAIN_DIRS = ['train_00', 'train_01', 'train_02', 'train_03',
              'train_04', 'train_05', 'train_06', 'train_07',
              'train_08']

sans_ext = lambda f: f[:-4]

image_ids_from = compose(list, map(sans_ext), flatten, map(compose(ls, partial(join, DIR))))

train_image_ids = image_ids_from(TRAIN_DIRS)
test_image_ids  = image_ids_from(['test'])
val_image_ids   = image_ids_from(['validation'])

def eye_filter_subset(name, df, ids):
    filtered_df = df[(df.LabelName == LABEL) & (df.ImageID.isin(ids))]
    print(name, filtered_df.shape)
    return filtered_df

In [22]:
train_ann_bbox = eye_filter_subset(
    'train_ann_bbox', train_ann_bbox, train_image_ids)
train_ann_label_box = eye_filter_subset(
    'train_ann_label_box', train_ann_label_box, train_image_ids)

val_ann_bbox = eye_filter_subset(
    'val_ann_bbox', val_ann_bbox, val_image_ids)
val_ann_label_box = eye_filter_subset(
    'val_ann_label_box', val_ann_label_box, val_image_ids)

test_ann_bbox = eye_filter_subset(
    'test_ann_bbox', test_ann_bbox, test_image_ids)
test_ann_label_box = eye_filter_subset(
    'test_ann_label_box', test_ann_label_box, test_image_ids)

train_ann_box (77233, 13)
train_ann_label_box (36272, 4)
val_ann_bbox (4304, 13)
val_ann_label_box (5077, 4)
test_ann_bbox (13034, 13)
test_ann_label_box (15373, 4)


In [25]:
train_ann_bbox.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
37,000004f4400f6ec5,xclick,/m/014sv8,1,0.477500,0.496875,0.399627,0.441643,1,0,0,0,0
38,000004f4400f6ec5,xclick,/m/014sv8,1,0.603125,0.630000,0.315593,0.338002,0,0,0,0,0
39,000004f4400f6ec5,xclick,/m/014sv8,1,0.659375,0.680000,0.319328,0.334267,0,0,0,0,0
122,0000271195f2c007,xclick,/m/014sv8,1,0.414062,0.509375,0.558333,0.633333,0,0,0,0,0
123,0000271195f2c007,xclick,/m/014sv8,1,0.559375,0.634375,0.572917,0.643750,0,0,0,0,0


# Prepare Yolo V3 (Keras version) for Training 

https://github.com/qqwweee/keras-yolo3